# Create Playlists

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from spotify_toolbox import *
import pydash

/Users/cassie/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [13]:
#spotipy defned, will only return 50 tracks at a time
MAX_RESPONSE = 50 

#user defined, the total number of loops my function will go through to get track ids. 100*50 = 5000.
#increase or decrease to make queries run faster or hella slow.
MAX_OFFSET = 100 

---
# Some Deets

I created one function `create_playlist_by_query` that will make the perfect (debatable) playlist given many different inputs. Here are your choices currently:

* genre list
* start year
* end year
* track name search (return if track contains a certain word)

The API is not super amazing, so you first search by the above. THEN, once those are pulled down, we can filter by audio features or tempo (shown below). 


The 'load_saved_results' flag will either query spotify directly, or load previously saved tracks from the API search. ie, if you don't want it to take forever and have already ran the year/genre/atrist/album/track query, setting this to true will load the cached results. 

## Audio Features

These are the audio features spotify has that you can query on. Cool.

https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

* **Mode**: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0
* **Acousticness**: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. 
* **Danceability**: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
* **Energy**: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
* **Instramentalness**: Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. 
* **Liveness**: Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.                 
* **Loudness**: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. 
* **Speechiness**: Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
* **Valence**: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

* **Tempo**: the tempo. Obviously

    
    

---
# Create Playlists

In [4]:
def create_playlist_by_query(playlist_name,
                             num_tracks,
                             query_name, 
                             genre_list,
                             start_year,
                             end_year,
                             track_search_term,
                             load_saved_results=False,
                              **audio_features):
    
    '''
    Creates a playlist based on so many query filters. 
    
    inputs:
        playlist_name: What you want your playlist name ot be. Note - Spotify does not require unique playlist names.
        num_tracks: Number of tracks desired. Playlist tracks will be random if num_tracks is greater than the number
                    of tracks returned by the api query and audio feature filtering
        query_name: what the query will be saved as in the saved_queries folder
        genre_list: a list of genres to query
        start_year: obvs
        end_year: obvs
        track_search_term: a term that you want to appear in the title of each track. 
        load_saved_results: load cached track ids from the saved_queries folder. Will look for the query saved as query_name
        audio features: a dictionary containing any or none of 'tempo_range',
                                                                'acousticness_range',
                                                                'danceability_range',
                                                                'energy_range',
                                                                'instrumentalness_range',
                                                                'liveness_range',
                                                                'loudness_range',
                                                                'speechiness_range',
                                                                'valence_range',
                                                                'popularity_range'
                                                                
        warning - many combinations of these paramters will result in very small playlists.
        
    outputs:
        tracks_df: dataframe of all tracks returned. Useful for data-mining and other fun times.

    '''
    
    if load_saved_results:
        try:
            print(f'loading results from {query_name}.pkl')
            track_df = load_pickle(os.path.join('saved_queries',f'{query_name}.pkl'))
        
        except FileNotFoundError:
            print(f'saved query {query_name}.pkl not found')
            load_saved_results = False
            
    if not load_saved_results:
        track_df = query_for_tracks(genre_list, start_year, end_year, track_search_term)
        print(f'saving query results to {query_name}.pkl')
        save_pickle(track_df, os.path.join('saved_queries',f'{query_name}.pkl'))
    
    if len(track_df) > 0:
        if audio_features:
            track_df = filter_for_audio_features(track_df, **audio_features)
        track_ids = list(track_df.id.values)
        track_ids = list(np.unique(track_ids))
        
    print(f'{len(track_ids)} total tracks found')
    tracks = random.sample(track_ids, min(num_tracks, len(track_ids)))         
    playlist_id = create_playlist(playlist_name)
    sp.user_playlist_add_tracks(user=username, playlist_id=playlist_id, tracks=tracks)
    
    return track_df

## Read in a list (maybe not exhaustive?) of all the Spotify genres

In [7]:
with open('all_spotify_genres.txt') as f:
    all_genres = f.read().split('\n')

In [22]:
#I think these are sorted by popularity??
#print(all_genres)

In [23]:
genre_list = [genre for genre in all_genres if 'pop' in genre]

In [24]:
# so many
#print(genre_list)

---
# Example Queries

In [22]:
%%time
genre_list = ['pop',
              'dance pop',
              'post-teen pop',
              'pop rap',
              'pop rock',
              'electropop',
              'indie pop',
              'viral pop',
              'indie poptimism',
              'hip pop',
              'power-pop']

start_year = 2015
end_year = 2020
track_search_term = None

audio_features = {'tempo_range': (80,130),
                'acousticness_range': (0,1),
                'danceability_range': (0,1),
                'energy_range': (0,1),
                'instrumentalness_range':(0,1),
                'liveness_range': (0,1),
                'loudness_range': (-15,10),
                'speechiness_range': (0,1),
                'valence_range': (.5,1),
                'popularity_range': (50,100)}

playlist_name = 'happy and poppy'
query_name = 'pop'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=False,
                                     **audio_features)

saving query results to pop.pkl
59 total tracks found
Creating playlist "happy and poppy"
CPU times: user 17.9 s, sys: 1.67 s, total: 19.5 s
Wall time: 2min 47s


The result: https://open.spotify.com/playlist/0P1eYEIhTq5b8aEcxXrpZ6?si=7TRwCFoCSJaBgtBi8666pQ

In [37]:
genre_list = [genre for genre in all_genres if any(word in genre for word in ['edm','trap','electro'])][:10]
print(genre_list)

start_year = 2015
end_year = 2020
track_search_term = None

audio_features = {'tempo_range': (120,220),
                'acousticness_range': (0,1),
                'danceability_range': (.5,1),
                'energy_range': (.5,1),
                'instrumentalness_range':(0,1),
                'liveness_range': (0,1),
                'loudness_range': None,
                'speechiness_range': (0,1),
                'valence_range': (0,1),
                'popularity_range': (0,100)}

playlist_name = 'ultimate party time'
query_name = 'party'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=True,
                                     **audio_features)

['trap', 'edm', 'electropop', 'electro house', 'trap latino', 'pop edm', 'vapor trap', 'progressive electro house', 'electronica', 'dark trap']
loading results from party.pkl
271 total tracks found
Creating playlist "ultimate party time"


The result: https://open.spotify.com/playlist/5d5Tzr0SlGVVCGhpWJ0oTv?si=9ScmKZVVSRKMi8ndySc6Tg

In [55]:
#Create a playlist like other artists
genres_like = ['grimes', 'M83', 'LCD Soundsystem']
responses = [sp.search(q=f'artist:{artist}', type='artist', limit=50) for artist in genres_like]
genre_list = list(set(pydash.flatten([response['artists']['items'][0]['genres'] for response in responses])))
print(genre_list)

start_year = 2005
end_year = 2020
track_search_term = None

audio_features = {'tempo_range': (50,120),
                'acousticness_range': (.5,1),
                'danceability_range': (0,.5),
                'energy_range': (0,1),
                'instrumentalness_range':(0,.5),
                'liveness_range': (0,1),
                'loudness_range': None,
                'speechiness_range': (0,1),
                'valence_range': (0,1),
                'popularity_range': (0,100)}

playlist_name = 'grimesy'
query_name = 'grimesy'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=False,
                                     **audio_features)

['electropop', 'alternative rock', 'escape room', 'electronica', 'chillwave', 'new rave', 'grave wave', 'electronic rock', 'modern rock', 'dance-punk', 'alternative dance', 'garage rock', 'canadian electropop', 'metropopolis', 'art pop', 'indietronica', 'indie pop', 'neo-synthpop', 'indie rock', 'dance pop', 'french shoegaze', 'rock']
saving query results to grimesy.pkl
33 total tracks found
Creating playlist "grimesy"


The result: https://open.spotify.com/playlist/5USuDe1PkUu1MOyM58X1mn?si=ZlKowzaMRHiN6UMzXhqC0w

## Tempo 

In [21]:
genre_list = all_genres[:15]
print(genre_list)

start_year = 1900
end_year = 2020
track_search_term = None

['pop', 'dance pop', 'rap', 'post-teen pop', 'pop rap', 'rock', 'latin', 'hip hop', 'trap', 'modern rock', 'edm', 'pop rock', 'tropical house', 'reggaeton', 'melodic rap']


In [11]:
audio_features = {'tempo_range': (65,75)}

playlist_name = 'The AI made me do it - 65 to 75'
query_name = 'tempo'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=False,
                                     **audio_features)

saving query results to tempo.pkl
25 total tracks found
Creating playlist "The AI made me do it - 65 to 75"


https://open.spotify.com/playlist/5jbjaP4uUa1wnmE4BSKyK9?si=WUzVRIQjQne_SQU3Uw4cfQ

In [16]:
audio_features = {'tempo_range': (75,85)}

playlist_name = 'The AI made me do it - 75 to 85'
query_name = 'tempo'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=True,
                                     **audio_features)

loading results from tempo.pkl
125 total tracks found
Creating playlist "The AI made me do it - 75 to 85"


https://open.spotify.com/playlist/6NMQQi7lsDbufKGhIwFjna?si=LgdiCnZJSZ2EJnuRX6aaJQ

In [17]:
audio_features = {'tempo_range': (85,95)}

playlist_name = 'The AI made me do it - 85 to 95'
query_name = 'tempo'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=True,
                                     **audio_features)

loading results from tempo.pkl
201 total tracks found
Creating playlist "The AI made me do it - 85 to 95"


https://open.spotify.com/playlist/729x7EAiKizHatHzAjIbEg?si=8qJzyAtpRcmTM02NoHDXag

In [18]:
audio_features = {'tempo_range': (95,105)}

playlist_name = 'The AI made me do it - 95 to 105'
query_name = 'tempo'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=True,
                                     **audio_features)

loading results from tempo.pkl
285 total tracks found
Creating playlist "The AI made me do it - 95 to 105"


https://open.spotify.com/playlist/5IIjW3oK3YjGKuvnMVBrUb?si=KrB6PjEMTkKsxcfNo6ByfQ

In [20]:
audio_features = {'tempo_range': (105,115)}

playlist_name = 'The AI made me do it - 105 to 115'
query_name = 'tempo'

tracks_df = create_playlist_by_query(playlist_name=playlist_name,
                                     num_tracks=50,
                                     query_name=query_name,
                                     genre_list=genre_list,
                                     start_year=start_year,
                                     end_year=end_year,
                                     track_search_term=track_search_term,
                                     load_saved_results=True,
                                     **audio_features)

loading results from tempo.pkl
172 total tracks found
Creating playlist "The AI made me do it - 105 to 115"


https://open.spotify.com/playlist/4lFZgDWyCjgrzrpmX08E9n?si=BbFSRFg0THalz6igjxsLsg